In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss, f1_score
import numpy as np
import datetime as dt

In [ ]:
playa_att_df = pd.read_csv('../data/playa_nogeometry_whucs.csv')
playa_att_df = playa_att_df.set_index('id')
playa_att_df = playa_att_df[['state','acres','countyfips','cluster','farmed','hydromod','healthy', 'huc12', 'author']]
playa_att_df['huc4'] = playa_att_df['huc12'].astype(str).str[0:4]

In [ ]:
pred_df = pd.read_csv('../data/all_preds.csv')

In [ ]:
# Add months and years back in
pred_df['date'] = np.tile(pd.date_range('1984-03', periods=418, freq='M'), int(pred_df.shape[0]/418))

In [ ]:
pred_df = pred_df.set_index(['id', 'date'])

# Calculate loss per playa

In [ ]:
# Looping to save some memory
per_loop = 2000
starts = np.arange(0, pred_df.shape[0], per_loop*418)

In [ ]:
max_inun = pred_df.groupby('id').aggregate({'true':'max'})
nonzero_ids = max_inun.index[max_inun['true'].values>0]

In [ ]:
per_playa_loss_series = pd.Series(dtype='float64')
for start_point in starts:
    end = min(pred_df.shape[0], start_point + per_loop*418)
    temp_loss_series = pred_df.iloc[start_point:end].groupby('id').apply(lambda df: log_loss(df['true'],df['pred'], labels=[0,1]))
    per_playa_loss_series = per_playa_loss_series.append(temp_loss_series)

In [ ]:
nonzero_per_playa_loss = per_playa_loss_series.loc[nonzero_ids].sort_values()

In [ ]:
# Plot best, worst, and a few in the middle
def plot_single_playa_trajectory(df, id):
    playa_df = df.loc[id]
    playa_df[['true','pred']].plot(figsize=[15,5])

In [ ]:
plot_single_playa_trajectory(pred_df,nonzero_per_playa_loss.index[0])

In [ ]:
plot_single_playa_trajectory(pred_df, nonzero_per_playa_loss.index[-1])

In [ ]:
plot_single_playa_trajectory(pred_df, nonzero_per_playa_loss.index[int(nonzero_per_playa_loss.shape[0]/2)])
plot_single_playa_trajectory(pred_df, nonzero_per_playa_loss.index[int(nonzero_per_playa_loss.shape[0]/2)+1])
plot_single_playa_trajectory(pred_df, nonzero_per_playa_loss.index[int(nonzero_per_playa_loss.shape[0]/2)-1])

# Playa Inundation over time (predicted vs real)

In [ ]:
pred_df = pred_df.assign(pred_binary=pred_df['pred']>0.25)

In [ ]:
pred_df[['true','pred_binary']].groupby('date').mean().plot(figsize=[15,5])
plt.axvline(dt.datetime(2014,1,31), color='black')
plt.axvline(dt.datetime(2010,1,31), color='black')

# Spatial representation of val/test error

In [ ]:
def calc_f1_score(true, pred, cutoff=0.25):
    return f1_score(true, pred>cutoff)

In [ ]:
pred_df['set_flag'] = 2
pred_df.loc[pred_df.index.get_level_values(1)<pd.Timestamp('2014-01-01'), 'set_flag'] = 1
pred_df.loc[pred_df.index.get_level_values(1)<pd.Timestamp('2010-01-01'), 'set_flag'] = 0

In [ ]:
test_pred_df = pred_df.loc[pred_df['set_flag']==2]
val_pred_df = pred_df.loc[pred_df['set_flag']==1]

In [ ]:
log_loss(test_pred_df['true'], test_pred_df['pred'])

In [ ]:
log_loss(val_pred_df['true'], val_pred_df['pred'])

In [ ]:
test_pred_df = test_pred_df.reset_index().set_index('id')
val_pred_df = val_pred_df.reset_index().set_index('id')

In [ ]:
test_pred_df = test_pred_df.join(playa_att_df, how='inner')
val_pred_df = val_pred_df.join(playa_att_df, how='inner')

## By state to start

In [ ]:
state_test_loss = test_pred_df.groupby('state').apply(lambda df: log_loss(df['true'],df['pred'], labels=[0,1]))
state_val_loss = val_pred_df.groupby('state').apply(lambda df: log_loss(df['true'],df['pred'], labels=[0,1]))
state_test_f1 = test_pred_df.groupby('state').apply(lambda df: calc_f1_score(df['true'],df['pred']))
state_val_f1 = val_pred_df.groupby('state').apply(lambda df: calc_f1_score(df['true'],df['pred']))
state_count = playa_att_df.groupby('state').size()

In [ ]:
fig, ax = plt.subplots(2,3, figsize=[15,8])
state_count.plot.bar(ax=ax[0,0], title='Playa Count')
state_val_loss.plot.bar(ax=ax[0,1], title='Val Loss')
state_val_f1.plot.bar(ax=ax[0,2], title='Val F1')
state_test_loss.plot.bar(ax=ax[1,1], title='Test Loss')
state_test_f1.plot.bar(ax=ax[1,2], title='Test F1')
fig.tight_layout()

## By HUC 4

In [ ]:
huc_test_loss = test_pred_df.groupby('huc4').apply(lambda df: log_loss(df['true'],df['pred'], labels=[0,1]))
huc_val_loss = val_pred_df.groupby('huc4').apply(lambda df: log_loss(df['true'],df['pred'], labels=[0,1]))
huc_test_f1 = test_pred_df.groupby('huc4').apply(lambda df: calc_f1_score(df['true'],df['pred']))
huc_val_f1 = val_pred_df.groupby('huc4').apply(lambda df: calc_f1_score(df['true'],df['pred']))
huc_count = playa_att_df.groupby('huc4').size()

In [ ]:
fig, ax = plt.subplots(2,3, figsize=[15,8])
huc_count.plot.bar(ax=ax[0,0], title='HUC4 Playa Count')
huc_val_loss.plot.bar(ax=ax[0,1], title='Val Loss')
huc_val_f1.plot.bar(ax=ax[0,2], title='Val F1')
huc_test_loss.plot.bar(ax=ax[1,1], title='Test Loss')
huc_test_f1.plot.bar(ax=ax[1,2], title='Test F1')
fig.tight_layout()

## By Author

In [ ]:
author_test_loss = test_pred_df.groupby('author').apply(lambda df: log_loss(df['true'],df['pred'], labels=[0,1]))
author_val_loss = val_pred_df.groupby('author').apply(lambda df: log_loss(df['true'],df['pred'], labels=[0,1]))
author_test_f1 = test_pred_df.groupby('author').apply(lambda df: calc_f1_score(df['true'],df['pred']))
author_val_f1 = val_pred_df.groupby('author').apply(lambda df: calc_f1_score(df['true'],df['pred']))
author_count = playa_att_df.groupby('author').size()

In [ ]:
fig, ax = plt.subplots(2,3, figsize=[15,10])
author_count.plot.bar(ax=ax[0,0], title='Author Playa Count', rot=45)
author_val_loss.plot.bar(ax=ax[0,1], title='Val Loss', rot=45)
author_val_f1.plot.bar(ax=ax[0,2], title='Val F1', rot=45)
author_test_loss.plot.bar(ax=ax[1,1], title='Test Loss', rot=45)
author_test_f1.plot.bar(ax=ax[1,2], title='Test F1', rot=45)
fig.tight_layout()
